In [1]:
%load_ext rpy2.ipython

In [3]:
%%R
set.seed(123)
install.packages("dplyr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/dplyr_1.1.4.tar.gz'
Content type 'application/x-gzip' length 1207521 bytes (1.2 MB)
downloaded 1.2 MB


The downloaded source packages are in
	‘/tmp/RtmpDBGNOb/downloaded_packages’


In [4]:
%%R
install.packages("dplyr")
install.packages("haven")
install.packages("psych")
install.packages("lavaan")
install.packages("semTools")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/dplyr_1.1.4.tar.gz'
Content type 'application/x-gzip' length 1207521 bytes (1.2 MB)
downloaded 1.2 MB


The downloaded source packages are in
	‘/tmp/RtmpDBGNOb/downloaded_packages’
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/haven_2.5.5.tar.gz'
Content type 'application/x-gzip' length 316909 bytes (309 KB)
downloaded 309 KB


The downloaded source packages are in
	‘/tmp/RtmpDBGNOb/downloaded_packages’
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘mnormt’, ‘GPArotation’

trying URL 'https://cran.rstudio.com/src/contrib/mnormt_2.1.1.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/GPArotation_2025.3-1.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/psych_2.5.6.tar.gz'

The downloaded sour

In [5]:
%%R
library(dplyr)
library(haven)
library(psych)
library(lavaan)
library(semTools)


Attaching package: ‘psych’

The following objects are masked from ‘package:ggplot2’:

    %+%, alpha

This is lavaan 0.6-19
lavaan is FREE software! Please report any bugs.

Attaching package: ‘lavaan’

The following object is masked from ‘package:psych’:

    cor2cov

 
###############################################################################
This is semTools 0.5-7
All users of R (or SEM) are invited to submit functions or ideas for functions.
###############################################################################

Attaching package: ‘semTools’

The following objects are masked from ‘package:psych’:

    reliability, skew

The following object is masked from ‘package:readr’:

    clipboard



In [6]:
%%R
df <- read_sav("aiData.sav")

In [7]:
%%R
likert_vars <- c(paste0("A",1:5),
                 paste0("F",1:8),
                 paste0("TP",1:8),
                 paste0("TR",1:3),
                 paste0("U",1:6),
                 paste0("Q",1:5),
                 paste0("P",1:4),
                 paste0("AN",1:4))
stopifnot(all(likert_vars %in% names(df)))
df[likert_vars] <- lapply(df[likert_vars], function(x) as.numeric(x))

In [8]:
%%R
rev5 <- function(x) ifelse(is.na(x), NA, 6 - x)
df <- df |>
  mutate(
    A4_R  = rev5(A4),      # not used in final PACH
    F5_R  = rev5(F5),      # dropped from FAIR
    F6_R  = rev5(F6),      # dropped from FAIR
    AN1_R = rev5(AN1),
    AN2_R = rev5(AN2),
    AN3_R = rev5(AN3),
    AN4_R = rev5(AN4)
  )

In [9]:
%%R
# Blocks used for checks and HTMT
blocks <- list(
  PACH    = c("A1","A2","A3","A5"),
  FAIR    = c("F1","F2","F3","F4","F8"),
  TRANSP  = paste0("TP",1:8),
  TRUST   = paste0("TR",1:3),
  PU      = paste0("U",1:6),
  WE_QUAL = paste0("Q",1:5),
  WE_UTIL = paste0("P",1:4),
  WE_ANX  = c("AN1_R","AN2_R","AN3_R","AN4_R")
)
ordered_vars <- unique(unlist(blocks))

 --- 2) Measurement Models (CFA) -----------------------------------------

In [10]:
%%R
meas_model_base <- '
# First-order
PACH   =~ A1 + A2 + A3 + A5
FAIR   =~ F1 + F2 + F3 + F4 + F8
TRANSP =~ TP1 + TP2 + TP3 + TP4 + TP5 + TP6 + TP7 + TP8
TRUST  =~ TR1 + TR2 + TR3
PU     =~ U1 + U2 + U3 + U4 + U5 + U6
WE_QUAL =~ Q1 + Q2 + Q3 + Q4 + Q5
WE_UTIL =~ P1 + P2 + P3 + P4
WE_ANX  =~ AN1_R + AN2_R + AN3_R + AN4_R

# Second-order
WE =~ WE_QUAL + WE_UTIL
PE =~ FAIR + TRANSP + TRUST
'

Base CFA with MLR (robust)

In [11]:
%%R
fit_cfa0 <- cfa(meas_model_base, data = df, estimator = "MLR")
cat("\nBASE CFA fit (MLR):\n")
print(fitMeasures(fit_cfa0, c("cfi","tli","rmsea","srmr")))


BASE CFA fit (MLR):
  cfi   tli rmsea  srmr 
0.887 0.878 0.063 0.056 


# Auto-pick up to 3 within-factor residual covariances (MI >= 15)

In [12]:
%%R
mi_tbl <- modindices(fit_cfa0) |>
  as_tibble() |>
  filter(op == "~~", lhs != rhs) |>
  rowwise() |>
  mutate(factor = {
    f <- NA_character_
    for (nm in names(blocks)) {
      v <- blocks[[nm]]
      if (lhs %in% v && rhs %in% v) { f <- nm; break }
    }
    f
  }) |>
  ungroup() |>
  filter(!is.na(factor)) |>
  arrange(desc(mi))

mi_pick <- mi_tbl |>
  filter(mi >= 15) |>
  group_by(factor) |>
  slice_head(n = 1) |>
  ungroup() |>
  slice_head(n = 3) |>
  select(lhs, rhs, mi, epc, factor)

cat("\nCandidate residual pairs (max 3):\n"); print(mi_pick)


Candidate residual pairs (max 3):
# A tibble: 3 × 5
  lhs   rhs      mi   epc factor
  <chr> <chr> <dbl> <dbl> <chr> 
1 F1    F2    134.  0.570 FAIR  
2 A1    A2     34.0 0.200 PACH  
3 TP1   TP2    86.9 0.251 TRANSP


In [13]:
%%R
add_lines <- if (nrow(mi_pick) > 0) {
  paste0(mi_pick$lhs, " ~~ ", mi_pick$rhs, collapse = "\n")
} else { "" }

meas_model_final <- if (nchar(add_lines) > 0) {
  paste(meas_model_base, "\n# Added residual covariances (within-factor)\n", add_lines)
} else {
  meas_model_base
}

cat("\nAdded residual covariances:\n", add_lines, "\n")


Added residual covariances:
 F1 ~~ F2
A1 ~~ A2
TP1 ~~ TP2 


# FINAL CFA

In [14]:
%%R
fit_cfa <- cfa(meas_model_final, data = df, estimator = "MLR")
cat("\nFINAL CFA fit (MLR):\n")
print(fitMeasures(fit_cfa, c("cfi","tli","rmsea","srmr")))


FINAL CFA fit (MLR):
  cfi   tli rmsea  srmr 
0.919 0.912 0.054 0.054 


In [15]:
%%R
summary(fit_cfa, fit.measures = TRUE, standardized = TRUE)

lavaan 0.6-19 ended normally after 75 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                        96

  Number of observations                           341

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                              1357.411    1029.723
  Degrees of freedom                               684         684
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  1.318
    Yuan-Bentler correction (Mplus variant)                       

Model Test Baseline Model:

  Test statistic                              9039.960    6657.874
  Degrees of freedom                               741         741
  P-value                                        0.000       0.000
  Scaling correction factor                                  1.358

User 

# Reliability & AVE (from CFA)


In [16]:
%%R
cr <- compRelSEM(fit_cfa)
ave_vals <- AVE(fit_cfa)
cat("\nComposite Reliability:\n"); print(cr)


Composite Reliability:
   PACH    FAIR  TRANSP   TRUST      PU WE_QUAL WE_UTIL  WE_ANX 
  0.849   0.682   0.891   0.908   0.896   0.857   0.885   0.831 


In [17]:
%%R
cat("\nAVE:\n"); print(ave_vals)


AVE:
   PACH    FAIR  TRANSP   TRUST      PU WE_QUAL WE_UTIL  WE_ANX 
  0.626   0.357   0.535   0.766   0.588   0.539   0.653   0.566 


# --- 3) Discriminant validity (HTMT, polychoric) -------------------------
# IMPORTANT: Do HTMT on a *copy* with ordered factors — keep df numeric for SEM

In [18]:
%%R
df_htmt <- df
df_htmt[ordered_vars] <- lapply(df_htmt[ordered_vars], function(x) {
  if (is.ordered(x) || is.factor(x)) return(as.ordered(x))
  ordered(as.numeric(x), levels = sort(unique(na.omit(as.numeric(x)))))
})

In [19]:
%%R
meas_model_firstorder <- '
PACH   =~ A1 + A2 + A3 + A5
FAIR   =~ F1 + F2 + F3 + F4 + F8
TRANSP =~ TP1 + TP2 + TP3 + TP4 + TP5 + TP6 + TP7 + TP8
TRUST  =~ TR1 + TR2 + TR3
PU     =~ U1 + U2 + U3 + U4 + U5 + U6
WE_QUAL =~ Q1 + Q2 + Q3 + Q4 + Q5
WE_UTIL =~ P1 + P2 + P3 + P4
WE_ANX  =~ AN1_R + AN2_R + AN3_R + AN4_R
'

In [20]:
%%R
htmt_poly <- semTools::htmt(
  model   = meas_model_firstorder,
  data    = df_htmt,
  missing = "listwise",
  ordered = ordered_vars,
  absolute = TRUE,
  htmt2   = TRUE
)
cat("\nHTMT (polychoric, HTMT2):\n"); print(round(htmt_poly, 3))


HTMT (polychoric, HTMT2):
         PACH  FAIR TRANSP TRUST    PU WE_QUA WE_UTI WE_ANX
PACH    1.000                                              
FAIR    0.632 1.000                                        
TRANSP  0.540 0.708  1.000                                 
TRUST   0.786 0.700  0.594 1.000                           
PU      0.676 0.816  0.734 0.779 1.000                     
WE_QUAL 0.568 0.643  0.675 0.633 0.695  1.000              
WE_UTIL 0.528 0.570  0.584 0.598 0.691  0.797  1.000       
WE_ANX  0.032 0.086  0.056 0.040 0.082  0.117  0.052  1.000


# --- 5) Moderation: PACH x WE -> PE (product indicators) -----------------
# Build ALL product indicators (4 PACH items x 9 WE items = 36) on a copy

In [21]:
%%R
c0 <- function(v) v - mean(v, na.rm = TRUE)
pach_items <- blocks$PACH
we_items   <- c(blocks$WE_QUAL, blocks$WE_UTIL)

stopifnot(all(pach_items %in% names(df)), all(we_items %in% names(df)))
df_int <- df
prod_names_full <- character(0)
for (x in pach_items) {
  x_c <- c0(df_int[[x]])
  for (w in we_items) {
    w_c <- c0(df_int[[w]])
    nm <- paste0("pachWE_", x, "_", w)
    df_int[[nm]] <- c0(x_c * w_c)
    prod_names_full <- c(prod_names_full, nm)
  }
}

# --- 6) Harman one-factor CMV screen ------------------------------------


In [22]:
%%R
efa1 <- psych::fa(df[likert_vars], nfactors = 1, fm = "ml", rotate = "none")
cat("\nHarman one-factor % variance:\n")
print(efa1$Vaccounted["Proportion Var",1])


Harman one-factor % variance:
[1] 0.3303398


# --- 7) Controls: Gender & Job-search dummies ----------------------------
# Gender: 1=male, 2=female -> female=1, male=0

In [23]:
%%R
if (!"gender" %in% names(df)) stop("Missing 'gender' in the dataset.")
df$gender_female <- ifelse(df$gender == 2, 1,
                           ifelse(df$gender == 1, 0, NA))

# Emp grouping: 1–3 = not searching (0), 4–6 = searching (1)
if (!"Emp" %in% names(df)) stop("Missing 'Emp' in the dataset.")
df$job_search <- ifelse(df$Emp %in% 1:3, 0,
                        ifelse(df$Emp %in% 4:6, 1, NA))

In [24]:
%%R
cat("\nGender distribution  (Female/Male): ",
    sum(df$gender_female==1, na.rm=TRUE), "/",
    sum(df$gender_female==0, na.rm=TRUE), "\n", sep = "")


Gender distribution  (Female/Male): 199/142


In [25]:
%%R
cat("Job search (Not searching/Searching): ",
    sum(df$job_search==0, na.rm=TRUE), "/",
    sum(df$job_search==1, na.rm=TRUE), "\n", sep = "")

Job search (Not searching/Searching): 165/176


# --- 7a) Mediation with gender control -----------------------------------


In [26]:
%%R
set.seed(123)
sem_model_med_control <- paste0(meas_model_final, '

PU  ~ a1*PACH + g1*gender_female
PE  ~ c1*PACH + b1*PU + g2*gender_female

ind_PU := a1*b1
tot    := c1 + (a1*b1)
')

fit_med_control <- sem(sem_model_med_control, data = df,
                       estimator = "ML", se = "bootstrap", bootstrap = 5000)


In addition: Warning message:
lavaan->lav_model_nvcov_bootstrap():  
   1634 bootstrap runs resulted in nonadmissible solutions. 


In [27]:
%%R
cat("\nMEDIATION + GENDER control fit:\n")
print(fitMeasures(fit_med_control, c("cfi","tli","rmsea","srmr")))


MEDIATION + GENDER control fit:
  cfi   tli rmsea  srmr 
0.907 0.900 0.056 0.073 


In [28]:
%%R
med_control_params <- subset(parameterEstimates(fit_med_control, standardized = TRUE, ci = TRUE,
                                                boot.ci.type = "bca.simple"),
                             lhs %in% c("PU","PE","ind_PU","tot") & op %in% c("~",":="))[
                               , c("lhs","op","rhs","est","se","pvalue","ci.lower","ci.upper","std.all")]
cat("\nMediation + gender control (unstd + std):\n"); print(med_control_params)


Mediation + gender control (unstd + std):
       lhs op           rhs   est    se pvalue ci.lower ci.upper std.all
48      PU  ~          PACH 0.649 0.069  0.000    0.525    0.797   0.746
49      PU  ~ gender_female 0.170 0.067  0.012    0.040    0.305   0.115
50      PE  ~          PACH 0.242 0.061  0.000    0.146    0.387   0.504
51      PE  ~            PU 0.304 0.073  0.000    0.180    0.471   0.552
52      PE  ~ gender_female 0.009 0.029  0.767   -0.052    0.066   0.011
106 ind_PU :=         a1*b1 0.197 0.046  0.000    0.121    0.306   0.412
107    tot :=    c1+(a1*b1) 0.439 0.077  0.000    0.297    0.602   0.916


# --- 7b) Moderation with gender control ----------------------------------
# Use the FULL product indicator set (df_int)

In [37]:
%%R
if (!"gender" %in% names(df_int)) stop("Missing 'gender' in the dataset.")
df_int$gender_female <- ifelse(df_int$gender == 2, 1,
                           ifelse(df_int$gender == 1, 0, NA))

# Emp grouping: 1–3 = not searching (0), 4–6 = searching (1)
if (!"Emp" %in% names(df_int)) stop("Missing 'Emp' in the dataset.")
df_int$job_search <- ifelse(df_int$Emp %in% 1:3, 0,
                        ifelse(df_int$Emp %in% 4:6, 1, NA))


In [38]:
%%R
cat("\nGender distribution  (Female/Male): ",
    sum(df_int$gender_female==1, na.rm=TRUE), "/",
    sum(df_int$gender_female==0, na.rm=TRUE), "\n", sep = "")


Gender distribution  (Female/Male): 199/142


In [39]:
%%R
cat("Job search (Not searching/Searching): ",
    sum(df_int$job_search==0, na.rm=TRUE), "/",
    sum(df_int$job_search==1, na.rm=TRUE), "\n", sep = "")

Job search (Not searching/Searching): 165/176


In [40]:
%%R
set.seed(123)
sem_model_mod_control <- paste0(meas_model_final, '

INT =~ ', paste(prod_names_full, collapse = " + "), '

PU  ~ a1*PACH + g1*gender_female
PE  ~ c1*PACH + b1*PU + m1*INT + g2*gender_female

ind_PU := a1*b1
')


In [ ]:
%%R
fit_mod_control <- sem(sem_model_mod_control, data = df_int,
                       estimator = "ML", se = "bootstrap", bootstrap = 5000)

In [ ]:
%%R
cat("\nMODERATION + GENDER control fit:\n")
print(fitMeasures(fit_mod_control, c("cfi","tli","rmsea","srmr")))

In [ ]:
%%R
mod_control_params <- subset(parameterEstimates(fit_mod_control, standardized = TRUE, ci = TRUE,
                                                boot.ci.type = "bca.simple"),
                             lhs == "PE" & op == "~" &
                               rhs %in% c("PACH","PU","INT","gender_female"))[
                                 , c("lhs","op","rhs","est","se","pvalue","ci.lower","ci.upper","std.all")]
cat("\nPE equation (moderation + gender control):\n"); print(mod_control_params)

# --- 7c) Multi-group by job_search (UNLABELED = configural) --------------
## --- CONFIGURAL MULTI-GROUP (paths free across groups) --------------------

In [ ]:
%%R
if (!"gender_female" %in% names(df)) {
  if (!"gender" %in% names(df)) stop("Missing 'gender' in the dataset.")
  df$gender_female <- ifelse(df$gender == 2, 1,
                             ifelse(df$gender == 1, 0, NA))
}
if (!"job_search" %in% names(df)) {
  if (!"Emp" %in% names(df)) stop("Missing 'Emp' in the dataset.")
  # 1–3 = not searching (0), 4–6 = searching (1)
  df$job_search <- ifelse(df$Emp %in% 1:3, 0,
                          ifelse(df$Emp %in% 4:6, 1, NA))
}

In [ ]:
%%R
set.seed(123)

# Structural model with NO labels (so parameters are free and can differ by group)
mg_model <- paste0(meas_model_final, '
  # Structural (configural – paths free)
  PU ~ PACH + gender_female
  PE ~ PACH + PU + gender_female
')

fit_cfg <- sem(mg_model, data = df, group = "job_search",
               estimator = "ML", se = "bootstrap", bootstrap = 5000)

In [ ]:
%%R
cat("\nMULTI-GROUP (configural; paths free) fit:\n")
print(fitMeasures(fit_cfg, c("cfi","tli","rmsea","srmr")))

# Group-specific paths with bootstrap CIs and standardized estimates


In [ ]:
%%R
paths_tbl <- parameterEstimates(fit_cfg, standardized = TRUE, ci = TRUE,
                                boot.ci.type = "bca.simple") %>%
  filter(op == "~",
         lhs %in% c("PU","PE"),
         rhs %in% c("PACH","PU","gender_female")) %>%
  select(group, lhs, op, rhs, est, se, pvalue, ci.lower, ci.upper, std.all)

cat("\nGroup-specific structural paths (unstd & std):\n"); print(paths_tbl)

# ---- Compute indirect (a*b) and total (c' + a*b) per group after fitting ----


In [ ]:
%%R
get_indirects <- function(fit, standardized = FALSE) {
  pe <- parameterEstimates(fit, standardized = standardized)
  ng <- lavInspect(fit, "ngroups")
  out <- lapply(seq_len(ng), function(g) {
    a   <- pe$est[pe$group==g & pe$lhs=="PU" & pe$op=="~" & pe$rhs=="PACH"]   # PU ~ PACH
    b   <- pe$est[pe$group==g & pe$lhs=="PE" & pe$op=="~" & pe$rhs=="PU"]     # PE ~ PU
    cdp <- pe$est[pe$group==g & pe$lhs=="PE" & pe$op=="~" & pe$rhs=="PACH"]   # direct c'
    data.frame(group = g-1, ind_PU = a*b, total = cdp + a*b)
  })
  bind_rows(out)
}

ind_unstd <- get_indirects(fit_cfg, standardized = FALSE)
ind_std   <- get_indirects(fit_cfg, standardized = TRUE)

In [ ]:
%%R
cat("\nIndirects per group (unstd):\n"); print(ind_unstd)

In [ ]:
%%R
cat("\nIndirects per group (std):\n");   print(ind_std)

# ---- Optional: test equality of regressions across groups (LRT) -----------


In [ ]:
%%R
fit_equal_reg <- sem(mg_model, data = df, group = "job_search",
                     group.equal = "regressions",
                     estimator = "ML", se = "bootstrap", bootstrap = 2000)

cat("\nLRT: free vs. equal regressions across groups:\n")

In [ ]:
%%R
print(anova(fit_cfg, fit_equal_reg))

# Get descriptive statistics


In [ ]:
%%R
key_vars <- c("PACH_mean", "FAIR_mean", "TRANSP_mean", "TRUST_mean",
              "PU_mean", "WE_QUAL_mean", "WE_UTIL_mean")

In [ ]:
%%R
# Create means for each construct (if not already done)
df$PACH_mean <- rowMeans(df[c("A1", "A2", "A3", "A5")])
df$FAIR_mean <- rowMeans(df[c("F1", "F2", "F3", "F4", "F8")])
df$TRANSP_mean <- rowMeans(df[paste0("TP", 1:8)])
df$TRUST_mean <- rowMeans(df[paste0("TR", 1:3)])
df$PU_mean <- rowMeans(df[paste0("U", 1:6)])
df$WE_QUAL_mean <- rowMeans(df[paste0("Q", 1:5)])
df$WE_UTIL_mean <- rowMeans(df[paste0("P", 1:4)])

# Get descriptive statistics and correlation matrix
desc_stats <- describe(df[key_vars])
cor_matrix <- cor(df[key_vars], use = "complete.obs")

In [ ]:
%%R
print(desc_stats)

In [ ]:
%%R
print(round(cor_matrix, 2))